In [6]:
import pandas as pd

In [7]:
import psycopg2

try: 
    conn=psycopg2.connect(
       host='sumar.larioja.gob.ar',
       user='postgres',
       password='sumarLR21',
       database='sumar'
       )
    
    print("conexion exitosa")
    cursor = conn.cursor()
    cursor.execute("SELECT version()")
    row=cursor.fetchone()
    print(row)
    
except Exception as ex:
    print(ex)

conexion exitosa
('PostgreSQL 9.0.1, compiled by Visual C++ build 1500, 32-bit',)


In [8]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Consulta SQL para obtener la lista de esquemas
query = "SELECT schema_name FROM information_schema.schemata"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
esquemas = cursor.fetchall()

# Imprimir los esquemas
print("Lista de esquemas:")
for esquema in esquemas:
    print(esquema[0])

# Cerrar el cursor y la conexión
cursor.close()


Lista de esquemas:
pg_toast
pg_temp_1
pg_toast_temp_1
pg_catalog
information_schema
alta
calidad
capacitacion
cardiopatia
centro
contabilidad
expediente
facturacion
fichero
general
leche
mensajes
monitoreo
nacer
nomenclador
permisos
personal
planillas
pss
public
puco
sistema
tareas_divisionsoft
trazadoras
uad
laravel


In [9]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Esquema específico que deseas consultar
esquema_especifico = "contabilidad"

# Consulta SQL para obtener las tablas en un esquema específico
query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{esquema_especifico}'"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
tablas = cursor.fetchall()

# Imprimir las tablas
print(f"Tablas en el esquema '{esquema_especifico}':")
for tabla in tablas:
    print(tabla[0])

# Cerrar el cursor y la conexión
cursor.close()


Tablas en el esquema 'contabilidad':
saldo
detalle_inciso
incentivo
egreso
inciso
ingreso


In [10]:
# Consulta SQL para obtener los datos de una tabla en un esquema específico
query = """
SELECT  e.cuie, 
				e.fecha_egreso AS Perido_Rendicion,
				i.nombre AS GASTO,
				inc.ins_nombre AS CATEGORIA,
				e.factura_compra, 
				CAST(e.monto_egreso AS INTEGER),
				e.comentario
FROM "contabilidad"."egreso" AS e
LEFT JOIN contabilidad.detalle_inciso AS i
ON e.id_gasto = i."id"
LEFT JOIN "contabilidad"."inciso" AS inc
ON e.id_inciso = inc.id_inciso
WHERE e.fecha_egreso > '2020-01-01'
ORDER BY monto_egreso DESC
"""
# Crear un DataFrame a partir de la consulta
df_egreso = pd.read_sql_query(query, conn)

# Imprimir el DataFrame
print(df_egreso)

# Cerrar la conexión
cursor.close()
conn.close()

C:\Users\FRANCISCO\AppData\Local\Temp\ipykernel_16068\4035201084.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_egreso = pd.read_sql_query(query, conn)


         cuie perido_rendicion  \
0      L00004       2023-05-29   
1      L94921       2022-12-30   
2      L03857       2022-07-22   
3      L94921       2023-04-11   
4      L00004       2023-04-26   
...       ...              ...   
11957  L09094       2020-10-01   
11958  L03927       2021-08-02   
11959  L03922       2022-06-01   
11960  L03938       2021-01-01   
11961  L03929       2021-01-01   

                                                   gasto  \
0      INSUMOS VARIOS                                ...   
1      EQUIPAMIENTO MEDICO VARIOS                    ...   
2      INSUMOS MEDICOS VARIOS                        ...   
3      PC                                            ...   
4      INSUMOS MEDICOS VARIOS                        ...   
...                                                  ...   
11957  GASTOS Y COMISIONES BANCARIAS                 ...   
11958  GASTOS Y COMISIONES BANCARIAS                 ...   
11959  GASTOS Y COMISIONES BANCARIAS               

In [11]:
df_egreso.head()

,cuie,perido_rendicion,gasto,categoria,factura_compra,monto_egreso,comentario
0,L00004,2023-05-29,INSUMOS VARIOS ...,INSUMOS - Otros,00005-00000064,9935640,40 Baterias hrl 12280w 12vol 75amp- Montaje de...
1,L94921,2022-12-30,EQUIPAMIENTO MEDICO VARIOS ...,INVERSIONES - Equipamiento Medico,00002-00000578,9546000,EQ500maf - Equipo de rayos xdinar modelo 500 m...
2,L03857,2022-07-22,INSUMOS MEDICOS VARIOS ...,INSUMOS - Medicos,0004-00002618,7774114,Torre de videoendoscopia Olympus - Empr. Bioan...
3,L94921,2023-04-11,PC ...,INVERSIONES - Equipamiento No Medico,00008-00000286,5543802,CPU Y MONITOR MARCA GFAST MAS UPS MARCA LYONN
4,L00004,2023-04-26,INSUMOS MEDICOS VARIOS ...,INSUMOS - Medicos,0203-00000005,5168749,Wiener labcm 250 c/cód de barra


In [12]:
df_egreso['perido_rendicion'] = pd.to_datetime(df_egreso['perido_rendicion'])

In [13]:
df_egreso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11962 entries, 0 to 11961
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cuie              11962 non-null  object        
 1   perido_rendicion  11962 non-null  datetime64[ns]
 2   gasto             11921 non-null  object        
 3   categoria         11962 non-null  object        
 4   factura_compra    11962 non-null  object        
 5   monto_egreso      11962 non-null  int64         
 6   comentario        11962 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 654.3+ KB


In [14]:
df_egreso.to_csv('tablas_powerBI/df_egreso.csv', index=False)

